In [6]:
epochs = 200

In [7]:
import torch
from torch.nn import CrossEntropyLoss
from lambeq import BobcatParser
import numpy as np

train_data = ['cat runs on land']
train_labels = [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]

parser = BobcatParser()

train_diagrams = parser.sentences2diagrams(train_data)

loss = CrossEntropyLoss()

def CEL(y_hat, y):
    flattened_y_hat = y_hat.flatten()
    return float(loss(torch.tensor(flattened_y_hat), torch.tensor(y[0])))


def acc(y_hat, y):
    a=y_hat.flatten()
    #b=y_hat
    max_index = np.argmax(a)
    new_arr = np.zeros(a.shape)
    new_arr[max_index] = 1
    if np.array_equal(new_arr, y[0]):
        return 1
    else:
        return 0

from lambeq import AtomicType, IQPAnsatz

N = AtomicType.NOUN
S = AtomicType.SENTENCE
P = AtomicType.PREPOSITIONAL_PHRASE
ansatz = IQPAnsatz(ob_map={N: 1, S: 4, P:1}, n_layers=3, n_single_qubit_params=6)
train_circuits = [ansatz(diagram) for diagram in train_diagrams]


from lambeq import NumpyModel
model = NumpyModel.from_diagrams(train_circuits)
from lambeq import QuantumTrainer, SPSAOptimizer

# here you can play around with hyperparameters
trainer = QuantumTrainer(
    model,
    loss_function=CEL,
    epochs=epochs,
    optimizer=SPSAOptimizer,
    optim_hyperparams={'a': 0.2, 'c': 0.06, 'A':0.01*epochs},
    evaluate_functions={'acc': acc},
    evaluate_on_train=True,
    verbose = 'text',
    seed=0
)

from lambeq import Dataset

train_dataset = Dataset(
            train_circuits,
            train_labels,
            batch_size=1,
            shuffle=False)


Tagging sentences:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing tagged sentences:   0%|          | 0/1 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
trainer.fit(train_dataset, logging_step=1)

Epoch 1:    train/loss: 2.8305   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 2:    train/loss: 2.8097   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 3:    train/loss: 2.8107   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 4:    train/loss: 2.8019   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 5:    train/loss: 2.7670   valid/loss: -----   train/acc: 1.0000   valid/acc: -----
Epoch 6:    train/loss: 2.6856   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 7:    train/loss: 2.7324   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 8:    train/loss: 2.6878   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 9:    train/loss: 2.6023   valid/loss: -----   train/acc: 0.0000   valid/acc: -----
Epoch 10:   train/loss: 2.6023   valid/loss: -----   train/acc: 1.0000   valid/acc: -----
Epoch 11:   train/loss: 2.5546   valid/loss: -----   train/acc: 1.0000   valid/acc: -----
Epoch 12: 

In [12]:
model.forward(train_circuits).flatten()

array([0.12055281, 0.61578376, 0.00976868, 0.03333487, 0.0007148 ,
       0.01370844, 0.00289223, 0.06434937, 0.01203994, 0.10307202,
       0.00219166, 0.00586063, 0.00156769, 0.00437874, 0.00363166,
       0.0061527 ])

In [4]:
'''from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.get_backend("ibm_washington")'''

In [5]:
from pytket.extensions.qiskit import set_ibmq_config
set_ibmq_config(hub= "ibm-q-unibw", group="external-student", project="qnlp", ibmq_api_token="7c4a348dd83600545d560523c09e0cb05ce80191c267fcce23eba6e11cb8af4b5d446ab4182525881e9e3195a383a1006f92af6a4084d35b05e27f8456321d80")

In [15]:
import torch
from torch.nn import CrossEntropyLoss
from lambeq import BobcatParser
import numpy as np

train_data = ['cat runs on land']
train_labels = [[0., 1.]]

parser = BobcatParser()

train_diagrams = parser.sentences2diagrams(train_data)

loss = CrossEntropyLoss()

def CEL(y_hat, y):
    flattened_y_hat = y_hat.flatten()
    return float(loss(torch.tensor(flattened_y_hat), torch.tensor(y[0])))


def acc(y_hat, y):
    a=y_hat.flatten()
    #b=y_hat
    max_index = np.argmax(a)
    new_arr = np.zeros(a.shape)
    new_arr[max_index] = 1
    if np.array_equal(new_arr, y[0]):
        return 1
    else:
        return 0

from lambeq import AtomicType, IQPAnsatz

N = AtomicType.NOUN
S = AtomicType.SENTENCE
P = AtomicType.PREPOSITIONAL_PHRASE
ansatz = IQPAnsatz(ob_map={N: 1, S: 1, P:1}, n_layers=1, n_single_qubit_params=1)
train_circuits = [ansatz(diagram) for diagram in train_diagrams]

from pytket.extensions.qiskit import IBMQBackend, IBMQEmulatorBackend
from lambeq import TketModel
backend = IBMQBackend("ibmq_kolkata")
backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'shots': 8192
}
model = TketModel.from_diagrams(train_circuits, backend_config=backend_config)



from lambeq import QuantumTrainer, SPSAOptimizer

# here you can play around with hyperparameters
trainer = QuantumTrainer(
    model,
    loss_function=CEL,
    epochs=epochs,
    optimizer=SPSAOptimizer,
    optim_hyperparams={'a': 0.2, 'c': 0.06, 'A':0.01*epochs},
    evaluate_functions={'acc': acc},
    evaluate_on_train=True,
    verbose = 'text',
    seed=0
)

from lambeq import Dataset

train_dataset = Dataset(
            train_circuits,
            train_labels,
            batch_size=1,
            shuffle=False)


Tagging sentences:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing tagged sentences:   0%|          | 0/1 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
trainer.fit(train_dataset, logging_step=1)

ResultHandleTypeError: ResultHandle(<bound method JobV1.job_id of <RuntimeJob('cgvrmkfgbjvrfp7g2tu0', 'sampler')>>, 0, 9, 'null') does not match expected identifier types (<class 'str'>, <class 'int'>, <class 'int'>, <class 'str'>)